LSTM (장단기 메모리)
전에 했던 가장 기본이 되는 RNN은 time step이 길어지면 길어질 수록 앞의 정보가 충분하지 못하게 된다. 즉 다음 스텝으로 가면갈수록 옅어진다.
이런 문제를 장기 의존성 문제라고 한다.

LSTM은 이 문제를 해결하기 위해서 메모리셀에 입력게이트, 망각게이트, 출력게이트를 추가해서 필요없는건 지우고 중요한걸 남기게된다. 또한 셀 상태라는 값이 추가가된다. 
추가가 되는 게이트에는 전부 시그모이드가 존재하고, 과정은 다음과 같다.

1. 이전 time step에서 넘어오는 hidden state가 있을 때
2. 총 4번 연산을 진행한다.
3. 첫번째는 입력게이트에서 이전 상태에 곱해주는 가중치1, 그리고 현재 embedding을 변경시켜주는 가중치2 곱셈 합 계산 (시그모이드)
4. 두번째는 입력게이트에서 이전 상태에 곱해주는 가중치3, 그리고 현재 embedding을 변경시켜주는 가중치4 곱셈 합 계산 (하이퍼볼릭 탄젠트)
5. 세번째는 삭제게이트에서 이전 상태에 곱해주는 가중치5, 그리고 현재 embedding을 변경시켜주는 가중치6 곱셈 합 계산 (시그모이드)
6. 네번째는 셀상태(장기상태)에서 이전셀상태와 삭제게이트에서 계산된 셀상태 원소별 곱 + 입력게이트에서 계산된 두개 원소별 곱
7. 다섯번째는 출력게이트에서 이전 상태에 곱해주는 가중치7, 그리고 현재 embedding을 변경시켜주는 가중치8 곱셈 합 계산 (시그모이드)
8. 여섯번째는 출력게이트에서 계산된 셀 상태에 하이퍼볼릭 탄젠트한 것과 다섯번째에서 구한 값 원소별 곱

... 상당히 번거롭다 하지만 이렇게 하므로서 장기적으로 보낼 수 있게 된다.

In [1]:
#torch의 LSTM
import torch
import torch.nn as nn

inputs = torch.Tensor(1,10,5)
hidden_size = 8

LSTM = nn.LSTM(5,hidden_size,2,batch_first=True)
print(inputs)
output = LSTM(inputs)
print(output)

tensor([[[-4.9171e+04,  7.2027e-43,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]]])
(tensor([[[ 0.1037, -0.0426, -0.0065, -0.0355,  0.0836, -0.1027, -0.0394,
           0.0663],
         [ 0.1230, -0.1246, -0.0107,  0.0243,  0.1563, -0.1783, -0.1138,
           0.1064],
         [ 0.1161, -0.1767, -0.0122,  0.0735,  0.20

토큰화 !

크롤링 등을 통해서 얻어진 데이터가 전처리되어있지 않다면 꼭 진행해야함.
용도에 맞게 토큰화&정제&정규화 해야한다. 토큰의 기준은 다르지만 단어로 기준을 삼는다면 단어 토큰화가 된다.  

구두점과 같은 문자를 제외시키는 토큰화를 해보자

In [2]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
import nltk
#nltk.download()
print('단어 토큰화1 :',word_tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))

단어 토큰화1 : ['Do', "n't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr.', 'Jone', "'s", 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


In [3]:
print('단어 토큰화2 :',WordPunctTokenizer().tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))

단어 토큰화2 : ['Don', "'", 't', 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr', '.', 'Jone', "'", 's', 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


단어 토큰화 1에서는 '가 들어간 부분을 n't 와 's로 토큰화했지만 단어 토큰화 2에서는 '만 따로 토큰화 했다.

그러면 토큰화는 구두점이나 특수문자를 단순하게 분리해서 없애는 것일까?
-> 그것은 아니다 2016/03/02 나 AT&T와 같이 특수문자가 포함되어야지 의미가 있는 단어들도 많이 존재하기 때문이다.
-> New york 나 rock n roll 과 같이 띄어쓰기가 포함되어야지 하나의 단어가 되는 경우도 존재한다.

이를 처리하기 위해서 다양한 방법들이 고안되었지만 가장 많이 사용되는 방법으로 Penn Treebank Tokenizer가 있다.
이 방법의 규칙은 대략적으로 두가지이다.
1. 하이푼으로 구성된 단어는 하나로 유지한다.
2. don't와 같이 아포스트로피로 접어가 함께하는 단어는 분리한다.

실제로 어떻게 수행되나 보자

In [4]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

text = "Starting a home-based restaurant may be an ideal. it doesn't have a food chain or restaurant of their own."

print('트리뱅크 토크나이저 결과 : ',tokenizer.tokenize(text))

트리뱅크 토크나이저 결과 :  ['Starting', 'a', 'home-based', 'restaurant', 'may', 'be', 'an', 'ideal.', 'it', 'does', "n't", 'have', 'a', 'food', 'chain', 'or', 'restaurant', 'of', 'their', 'own', '.']


home-base는 하나의 단어로 n't는 따로 분리한 것을 확인할 수 있다. 

다음은 문장 토큰화이다. 
토큰의 단위가 문장일 경우 문장단위로 구분하는 과정을 문장 분류 (sentence segmentation)이라고 한다. 보통 가지고 있는 코퍼스가 정제되지 않았다면, 문장토큰화가 필요할 수 있다.

. 과 ! 와 같은 구분자로 구분하는데, .의 경우에는 상당히 애매하다. IP주소에는 .이 들어가고 이메일 주소에도 .이 들어가기 때문이다.
NLTK에서는 영어문장을 토큰화할 수 있는 sent_tokenize를 지원하고 있다.


In [5]:
from nltk.tokenize import sent_tokenize

text = "His barber kept his word. But keeping such a huge secret to himself was driving him crazy. Finally, the barber went up a mountain and almost to the edge of a cliff. He dug a hole in the midst of some reeds. He looked about, to make sure no one was near."
print('문장 토큰화 결과 : ',sent_tokenize(text))

문장 토큰화 결과 :  ['His barber kept his word.', 'But keeping such a huge secret to himself was driving him crazy.', 'Finally, the barber went up a mountain and almost to the edge of a cliff.', 'He dug a hole in the midst of some reeds.', 'He looked about, to make sure no one was near.']


성공적으로 처리된걸 확인할 수 있다. 하지만 중간중간에 . 이 들어가는 경우에는 어떻게 처리되나 확인해보자.

In [6]:
text = "I am actively looking for Ph.D. students. and you are a Ph.D student."
print('중간에 점이 들어간 경우 문장 토큰화 결과 : ',sent_tokenize(text))

중간에 점이 들어간 경우 문장 토큰화 결과 :  ['I am actively looking for Ph.D. students.', 'and you are a Ph.D student.']


sent_tokenize는 단순히 .을 구분자로 사용하지 않았기 때문에 성공적으로 인식한 것을 확인할 수 있다. 
한국어의 경우 KSS를 사용하자

In [7]:
import kss

text = '딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?'

print('한국어 문장 토큰화 : ',kss.split_sentences(text))

[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://cleancode-ws.tistory.com/97
- konlpy.tag.Mecab: https://uwgdqo.tistory.com/363



한국어 문장 토큰화 :  ['딥 러닝 자연어 처리가 재미있기는 합니다.', '그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다.', '이제 해보면 알걸요?']


한국어에서 토큰화의 어려움이 존재하는데, 영어의 경우 대부분 띄어쓰기 기준으로 토큰화를 진행하면 되지만 한국어의 경우 교착어(조사, 어미등을 붙여서 만드는 말)이기 때문에 상당히 번거롭다.
그를 그의 그에게 와 같이 다양한 조사가 문제가 된다. 이를 전부 분리해서 토큰화를 진행해줘야하기 때문에 한국어 토큰화에서는 형태소의 개념이 매우 중요하다.
또한 한국어의 경우 띄어쓰기에서도 문제가 발생한다.
영어는 단어 단위로 다 띄어버리면 되지만 한국어의 경우 띄어쓰기 규칙이 매우 복잡하다.

다음으로 토큰화 진행할 때 모든 언어에서 고려해야될 사항으로 품사가 있다. 우리나라 말로 따지면 '못'은 안된다의 의미와 망치와 함께 사용되는 것이 존재한다.
이를 처리해주는 것이 품사 태깅이다. (part of speech tagging)
NLTK와 KoNLPY 로 알아보자  

In [8]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

text = "I am actively looking for Ph.D. students. and you are a Ph.D. student."
tokenized_sentence = word_tokenize(text)

print('단어 토큰화 결과 : ',tokenized_sentence)
print('품사 태깅 결과 : ',pos_tag(tokenized_sentence))

단어 토큰화 결과 :  ['I', 'am', 'actively', 'looking', 'for', 'Ph.D.', 'students', '.', 'and', 'you', 'are', 'a', 'Ph.D.', 'student', '.']
품사 태깅 결과 :  [('I', 'PRP'), ('am', 'VBP'), ('actively', 'RB'), ('looking', 'VBG'), ('for', 'IN'), ('Ph.D.', 'NNP'), ('students', 'NNS'), ('.', '.'), ('and', 'CC'), ('you', 'PRP'), ('are', 'VBP'), ('a', 'DT'), ('Ph.D.', 'NNP'), ('student', 'NN'), ('.', '.')]


잘 수행된 것을 볼 수 있다.
PRP는 주어 VBP는 동사 RB는 부사 VGB는 현재부사 IN은 전치사 NNP는 고유명사 NNS는 복수형명사 CC는 접속사 DT는 관사를 의미한다.
한국어 자연어처리를 위해서는 KoNLPY를 자주 사용한다. 
형태소 분석기로 Okt , Kkma 등이 존재한다.

In [9]:
from konlpy.tag import Okt
from konlpy.tag import Kkma

okt = Okt()
kkma = Kkma()
text = "열심히 코딩한 당신, 연휴에는 여행을 가봐요"
print('Okt 형태소 분석 : ',okt.morphs(text))
print('Okt 품사 태깅 : ',okt.pos(text))
print('Okt 명사 추출 : ',okt.nouns(text))

Okt 형태소 분석 :  ['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
Okt 품사 태깅 :  [('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]
Okt 명사 추출 :  ['코딩', '당신', '연휴', '여행']


*konlpy의 경우에는 자바가 깔려있어야한다. 자바를 깔면서 환경변수도 설정해주면 정상적으로 작동하는 것을 확인할 수 있다. 

Okt의 morphs는 형태소를 분석하는 것이 있다.
pos는 품사를 태깅해주고, nouns는 명사를 추출해준다.
Kkma의 경우도 똑같이 사용가능하다.
다만 살짝 결과가 다르게 나온다. 
형태소 분석기는 이 두가지 이외에도 빠른 속도를 보여주는 메캅이라는게 존재한다. 자신이 짜는 것에 따라서 다르게 구성한다.

In [10]:
print('Kkma 형태소 분석 : ',kkma.morphs(text))
print('Kkma 품사 태깅 : ',kkma.pos(text))
print('Kkma 명사 추출 : ',kkma.nouns(text))

Kkma 형태소 분석 :  ['열심히', '코딩', '하', 'ㄴ', '당신', ',', '연휴', '에', '는', '여행', '을', '가보', '아요']
Kkma 품사 태깅 :  [('열심히', 'MAG'), ('코딩', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('당신', 'NP'), (',', 'SP'), ('연휴', 'NNG'), ('에', 'JKM'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('가보', 'VV'), ('아요', 'EFN')]
Kkma 명사 추출 :  ['코딩', '당신', '연휴', '여행']


코퍼스의 용도에 맞게 토큰을 분리하는 것을 토큰화라고 한다. 토큰화 작업 전과 후에는 텍스트 데이터를 용도에 맞게 정제 및 정규화 해야한다. 
정제 : 가지고 있는 코퍼스로부터 노이즈 데이터를 제거한다.
정규화 : 표현방법이 다른 단어들을 통합시켜서 같은 단어로 만들어준다.

정제작업은 토큰화를 더 정교하게 하기 위해서 토큰화 이전에 사용되기도 하지만, 토큰화 후에 노이즈를 제거하기 위해 사용되기도 한다.
완벽한 정제작업은 조금 힘들다.

규칙에 기반한 표기가 다른 단어들의 통합
USA와 US는 같은 뜻이고 uh-huh와 uhhuh는 같은 뜻이다. 이럴때 정규화 작업을 거치고 나면 US를 찾을 때 USA를 같이 찾을 수 있다.     
대표적인 방법으로는 어간 추출(stemming)과 표제어 추출(lemmatization)이 있다.

영어에서는 대,소문자 통합도 정규화가 된다. 그렇다고 전부 통합해버리면 US가 us(우리)가 되버릴 수 있기 때문에 구분되어야한다.


불필요한 단어들의 제거
정제작업의 일종으로 불용어제거, 등장 빈도가 적은 단어의 제거, 길이가 짧은 단어들을 제거하는 방법이 있다.


우선 표기가 다른 단어들을 통합하는 방법인 표제어 추출부터 연습해보자 !

In [11]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

print('표제어 추출 전 : ',words)
print('표제어 추출 후 : ',[lemmatizer.lemmatize(word) for word in words])

표제어 추출 전 :  ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']
표제어 추출 후 :  ['policy', 'doing', 'organization', 'have', 'going', 'love', 'life', 'fly', 'dy', 'watched', 'ha', 'starting']


위와 같이 lives 가 life dies가 dy has가 ha 처럼 변경된 것을 볼 수 있다. 일부는 틀리는데 이는 표제어 추출이 본래 단어의 품사를 알아야지 정확한 결과를 보여줄 수 있기 때문이다. 
아래와 같이 품사를 주면 정확하게 할 수 있고 품사가 보존된다.

In [12]:
print('품사를 알려줬을 때 표제어 추출',lemmatizer.lemmatize('lives','v'))

품사를 알려줬을 때 표제어 추출 live


어간추출(stemming)은 정해진 규칙만 보고 단어의 어미를 자르는 어림잡고 하는 작업이라고 볼 수 있다. 따라서 정확하지 않을 수 있고 사전에 존재하지 않는 단어일 수 있다.


In [14]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

text = "This was not the map we found in Billy Bones's chest, but an accurate copy, complete in all things--names and heights and soundings--with the single exception of the red crosses and the written notes."
tokenized = word_tokenize(text)
porter = PorterStemmer()

print('어간 추출 전 : ',tokenized)
print('어간 추출 후 : ',[porter.stem(word) for word in tokenized])

어간 추출 전 :  ['This', 'was', 'not', 'the', 'map', 'we', 'found', 'in', 'Billy', 'Bones', "'s", 'chest', ',', 'but', 'an', 'accurate', 'copy', ',', 'complete', 'in', 'all', 'things', '--', 'names', 'and', 'heights', 'and', 'soundings', '--', 'with', 'the', 'single', 'exception', 'of', 'the', 'red', 'crosses', 'and', 'the', 'written', 'notes', '.']
어간 추출 후 :  ['thi', 'wa', 'not', 'the', 'map', 'we', 'found', 'in', 'billi', 'bone', "'s", 'chest', ',', 'but', 'an', 'accur', 'copi', ',', 'complet', 'in', 'all', 'thing', '--', 'name', 'and', 'height', 'and', 'sound', '--', 'with', 'the', 'singl', 'except', 'of', 'the', 'red', 'cross', 'and', 'the', 'written', 'note', '.']


어간 추출은 표제어 추출에 비해서 속도가 빠르다는 장점이 있다. 영어 자연어 어간추출을 하려면 포터 어간 추출이 탁월한 선택이 될 수 있다.

불용어처리는 갖고있는 데이터에서 유의미한 단어 토큰만 선별하는 과정을 말한다. NLTK에서는 100여개의 불용어를 패키지 내부에 미리 설정해놨다.
개발자가 직접 설정할 수도 있다.

In [15]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt


stopword_list = stopwords.words('english')
print('불용어 갯수 : ',len(stopword_list))
print('불용어 10개 출력 : ',stopword_list[:10])

불용어 갯수 :  179
불용어 10개 출력 :  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [17]:
example = "Family is not an important thing. It's everything."
stop_words = set(stopword_list)

word_token = word_tokenize(example)

result = []

for word in word_token:
    if word not in stop_words:
        result.append(word)

print('불용어 제거 후 결과 : ',result)

불용어 제거 후 결과 :  ['Family', 'important', 'thing', '.', 'It', "'s", 'everything', '.']


한국어에서 불용어 제거
한국어에서는 간단하게 토큰화 이후 조사 접속사 등을 제거하는 것이 있다. 
또한 개발자가 개발하다가 추가하고 싶은 것들이 있을 수 있다. 이들을 처리할 수 있다.

In [18]:
okt = Okt()

example = "고기를 아무렇게나 구우려고 하면 안 돼. 고기라고 다 같은 게 아니거든. 예컨대 삼겹살을 구울 때는 중요한 게 있지."
example_token = okt.morphs(example)
stop_words = "를 아무렇게나 구 우려 고 안 돼 같은 게 구울 때 는"
stop_words = set(stop_words.split(' '))

result = []
for word in example_token:
    if word not in stop_words:
        result.append(word)

print('한국어 불용어 제거 후 결과 : ',result)

한국어 불용어 제거 후 결과 :  ['고기', '하면', '.', '고기', '라고', '다', '아니거든', '.', '예컨대', '삼겹살', '을', '중요한', '있지', '.']
